# Seaborn Review

Seaborn is a really great package for quickly producing nice plots. It's basic structure looks like this

![seaborn anatomy](images/seabornOverview.png)

The top row (the larger boxes) are figure-level plots. They handle much of the busywork of making a nice figure for you, and also allow you to select which underlying plot type to use (e.g. lineplot vs scatterplot). The underlying plots are axes-level plots. If you call them directly, they will return a matplotlib `axes` object, which you can then use to customize the plot. The topmost underlying plots are the default plot types for the figure-level plots.


There are also a couple functions, `pairplot()` and `jointplot()` that produce some common figures using a mix of plot types. We've made examples of both of these "by hand" already in this course.

Seaborn is built on top of matplotlib so, ultimately, everything in a seaborn figure is an `axes` or other matplotlib `artist`. This means that you can always use matplotlib methods if you need to do some low level customization of your figures.

## Preliminaries

First, let's import what we'll need:

In [ ]:
# This is all we should need in theory
import seaborn as sns

# But I had to do this to get plots to show for some random reason
import matplotlib.pyplot as plt
%matplotlib inline 

## Figure level plots

 We'll start with some figure level plots.

### Relational plots

Several example data sets come with seaborn. Here's one about tipping:

In [ ]:
tips = sns.load_dataset("tips")

Let's peek at the data set after loading (always!):

In [ ]:
tips

---

Use the cell below to `describe` the numerical data in `tips`:

Based on this summary of the numerical data in `tips`, do you think this is a recent US data set?

---

Count the number of smokers in `tips`.

Does this confirm or refute your guess about the origin of the data made on the numerical summary?

---

#### scatter plots

Let's make a call to `relplot`:

In [ ]:
sns.relplot(data=tips, x="total_bill", y="tip")

We can see that the vast majority of tips fall below a 20% line, so it probably is an old data set (assuming its from the US).

In terms of the plot *per se* though... so far, not a huge jump up from `matplotlib`...

Seaborn has five built in themes: `darkgrid` (the default), `whitegrid`, `dark`, `white`, and `ticks`. Let's set the default theme and replot.

In [ ]:
sns.set_theme()

In [ ]:
sns.relplot(data=tips, x="total_bill", y="tip")

That's better, and not unlike ggplot2's default theme in R!

Now, let's look at how easy it is to make a fancy plot by assigning other variables to plot aesthetic properties.

In [ ]:
sns.relplot(
    data=tips,
    x="total_bill", y="tip", col="time",
    hue="smoker", style="smoker", size="size",
)

Now that's a lot of plot for very little effort!

#### scatter plot with regression

Let's look at the tipping rate for females vs. males.

In [ ]:
sns.lmplot(data=tips, x="total_bill", y="tip", col="time", hue="sex")

Those look identical, so there is (was) gender equality in tipping at least. 

But let's do take a moment at what seaborn has done under the hood for us here. It has:

- run a linear regression on tip vs. bill separately for each group that we defined with our column and color specification
- plotted the regression lines
- computed 95% CIs on the fits, and plotted those as shaded areas.

Impressive. For those of you coming from R/tidyverse land, this is probably reminding you of ggplot() in more ways than the appearance of the background.

You might be wondering where we got `lmplot()`, as it's not one of the plots listed in the seaborn diagram above. Seaborn does have some "hidden gems" like `lmplot()` and `jointplot()` that you'll only discover by looking at the seaborn documentation.

Pro tip: There are two *really* useful places in the seaborn documentation to look for stuff:

- the [function interface reference](https://seaborn.pydata.org/api.html#function-interface)
- the [gallery](https://seaborn.pydata.org/examples/index.html)

In the [gallery](https://seaborn.pydata.org/examples/index.html), you can browse plot types by appearance; the mouse-over text will tell you the function used to make the plot, and clicking on the plot will take you to the example code! 

Now let's look at the tipping rate for smokers vs. non-smokers.

In [ ]:
sns.lmplot(data=tips, x="total_bill", y="tip", col="time", hue="smoker")

Hm. It looks like, at dinner at least, smokers may tip less (although the uncertainty on the fits is too high to be completely sure).

#### line plots

Scatter plots are usefully for pairwise data in which the pairs themselves are independent of one another. 

In other cases, the data are ordered by the x values. Often this is due to the y values unfolding over time. 

Let's load another data set to look at this.

In [ ]:
dots = sns.load_dataset("dots")

In [ ]:
# take a peek as always


The main data here are the firing rate of a neuron in the superior colliculus (a brain area crucial for moving the eyes) as a function of time. In the experiment, moving stimuli of various strengths ('coherence') appear on the screen, and the subject has to decide which way they moved (left or right, say). Later, the stimuli disappear and are replaced by two targets. The subject has to indicate their choice by moving their eyes to the appropriate target (left or right). Sometimes, the target corresponding to the subject's choice was within the neurons "area of responsibility" in the visual field (`choice` = `T1`), and sometimes without (`choice` = `T2`)

The neural recordings can either be aligned in time to the moment the stimuli appeared (`align` = `dots`) or to the precise moment the eye movement (a saccade) began (`align` = `sacc`)

Let's plot!

In [ ]:
sns.relplot(
    data=dots, kind="line",
    x="time", y="firing_rate", col="align",
    hue="choice", size="coherence", style="choice",
    facet_kws=dict(sharex=False),
)

Even if we knew nothing about the experiment, we could see that there is cool stuff going on. The plot makes it very clear that there is a huge effect of choice (eye movement direction), there is differential build-up of activity starting about 200 msec after the stimulus comes on, and that this neuron is almost certain involved in driving the eye movement in the T1 direction.

That's a lot of plot for very little effort. Notice that we didn't even have to remember to use different arguments for `size` and `style` when we switched from scatter to line plots – we used the same arguments and seaborn interpreted them as "line size" or "marker size", etc., as appropriate.

---

Re-make the above plot so that color codes stimulus strength and size codes choice.

Whether that made the plot better or worse is debatable, but the point is that we could so easily change it and see!

---

#### Automagic uncertainties (graphical stats)

Let's load another data set. This data set is from a functional MRI experiment.

In [ ]:
fmri = sns.load_dataset("fmri")

In [ ]:
# peek at the data
fmri

So we have a fairly large data set consisting of 5 variables:

- an fMRI signal
- time
- brain area
- type of event
- person

We can assign all 5 variables to aesthetic elements of a figure:

In [ ]:
sns.relplot(
    data=fmri, kind="line",
    x="timepoint", y="signal", col="region",
    hue="subject", style="event",
)

That worked, and an aficionado might be able to see what's going on. But the plot might be easier to interpret if we took the mean across subjects and just plot that (which was probably the whole point of running multiple subjects in this experiment).

Let's see what happens if we leave subject out of our aesthetic specification...

In [ ]:
sns.relplot(
    data=fmri, kind="line",
    x="timepoint", y="signal", col="region",
    hue="event", style="event",
)

Nice! The `relplot()` function figured out that, if we didn't a given variable explicitly coded in our plot, then we probably wanted to average across it. 

And because seaborn was written by an actual scientist, `lineplot()` (which we called above via the `kind` argument) also included 95% CIs on the mean computed by bootstrapping.

---

In the above plot, it looks like the standard deviation of activation might be proportional to mean activation. 

Check this quickly by just re-making the above plot, but having seaborn compute the standard deviation instead of the mean for us.

The doc page for `lineplot()` is [here](https://seaborn.pydata.org/generated/seaborn.lineplot.html#seaborn.lineplot).

Sure enough!

---

### Distribution plots

Seaborn makes it very easy to plot data distributions. Here's one for our fMRI data. The `kde` argument is going to add a 'kernel density estimate' (a best guess as to what the smooth version of the distribution looks like) for us.

In [ ]:
sns.displot(data=fmri, x="signal", col="event", kde=True)

That was easy but it's a little hard to compare these two distributions as is. Is overall activation higher for stimuli vs. cue? Or is overall activation the thing that matters?

---

Re-make the figure above so that visually comparing this distributions is easier. Hint: change the aesthetic property that 'event' maps to.

---

### Categorical plots

Let's try playing with `catplot()`

In [ ]:
sns.catplot(data=tips, x="day", y="total_bill", hue="smoker", alpha = 0.7)

A `stripplot` is the default axes-level plot for `catplot()` (and notice that the default axes-level plots are the first ones listed under their corresponding figure-level counterparts. But we can have it call `boxplot()` for us by telling it that we want `kind="box"`.

In [ ]:
sns.catplot(data=tips, kind="box", x="day", y="total_bill", hue="smoker")

If we request a bar plot, notice that seaborn computes and plots the means and also displays confidence intervals – another indication that seaborn was written by a scientist, not a programmer.

In [ ]:
sns.catplot(data=tips, kind="bar", x="day", y="total_bill", hue="smoker")

---

Make a violin plot showing signal (y axis) broken out by brain region and event type.

---

### Useful built in plots

Let's remake a plot similar to one we've made before.

First, we'll import numpy and pandas so we can make and store some data.

In [ ]:
import numpy as np
import pandas as pd

Then make some data like we did when reviewing matplotlib, but we'll convert to a pandas `DataFrame` at the end.

In [ ]:
### make some data to play with
my_means = [0, 0]
my_cov = [[2, -1.9], [-1.9, 3]]
my_n = 5000

my_rng = np.random.default_rng(42)
x, y = my_rng.multivariate_normal(my_means, my_cov, my_n).T

df = pd.DataFrame(dict(x=x, y=y))

And then we'll call the seaborn function `jointplot()` to make a "joint distribution plot".

In [ ]:
sns.jointplot(data=df, x='x', y='y', alpha = 0.2)

That was easy!

In order to really flex seaborn's muscles, though, let's load the built-in "penguins" data set.

In [ ]:
penguins = sns.load_dataset("penguins")

In [ ]:
penguins

So it looks like we have four measurements taken on penguins of different species, etc. Let's make a joint plot of some of these data.

In [ ]:
sns.jointplot(data=penguins, x="flipper_length_mm", y="body_mass_g", hue="species")

Ah, that's nice! We can see that body weight and flipper length are about as correlated as any biological measurement can get. Moreover, Gentoos are clearly biggest of the three types of dinosaur.

---

Re-make the above plot using bill length instead of body mass on the y axis.

What does this plot tell you? Could you discriminate amongst the species using either one of the variables alone? How about using both variables?

---

Make a `pairplot()` of the penguin data, using species to set the color.

In [ ]:
sns.pairplot(data=penguins, hue="species")

What combinations of variables would allow you to do a decent job of categorizing the species? Could you do it just based on bill measurements?

---

### matplotlib style customization

The axes-level functions return a matplotlib axes object. 

In [ ]:
ax = sns.scatterplot(data=df, x='x', y='y', alpha = 0.2)

We can verify this by checking its type.

In [ ]:
type(ax)

You can use this to customize your plot using the matplotlib axes methods, like `set_label()`, etc., just as though you had created the plot directly in matplotlib.

In [ ]:
ax = sns.scatterplot(data=df, x='x', y='y', alpha = 0.2)
ax.set_xlabel("Ex")
ax.set_ylabel("Why?")
ax.set_title('Why vs. Ex')

If we use one of the higher level plotting functions, we get back something called a FacetGrid that seaborn has created. 

In [ ]:
fg = sns.relplot(
    data=fmri, kind="line",
    x="timepoint", y="signal", col="region",
    hue="event", style="event",
)

In [ ]:
type(fg)

If we type "fg." and hit \<TAB\>, we can see that it has a lot of methods we can put to work for us.

In [ ]:
fg.

It also gives us access to the matplotlib figure via `fg.fig`, so we can set figure level properties.

---

In the cell below, add a line at the bottom to set the figure's background to a different color.

In [ ]:
fg = sns.relplot(
    data=fmri, kind="line",
    x="timepoint", y="signal", col="region",
    hue="event", style="event",
)

# set background to a different color



---

Here's a fancy example.

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.25)
fg = sns.relplot(
    data=penguins,
    x="bill_length_mm", y="bill_depth_mm", hue="body_mass_g",
    marker="o", s=100, alpha = 0.4
)
fg.set_axis_labels("Bill length (mm)", "Bill depth (mm)", labelpad=10)
fg.legend.set_title("Body mass (g)")
fg.figure.set_size_inches(6.5, 4.5)
fg.ax.margins(.15)
fg.despine(trim=True)

---

## Conclusion
Seaborn is a very powerful plotting package. It gives us both high-level easy plotting combined with the ability to do low-level customization if we wish. For custom figure layouts, matplotlib might be required. But for many plotting tasks, seaborn makes nice looking plots and can be a huge time saver!